In [1]:
import string

class Vectorizer:
    def standardize(self, text):
        text = text.lower()
        return "".join(char for char in text
                       if char not in string.punctuation)
        
    def tokenize(self, text):
        return text.split()
    
    def make_vocabulary(self, dataset):
        self.vocabulary = { "": 0, "[UNK]": 1 }
        for text in dataset:
            text = self.standardize(text)
            tokens = self.tokenize(text)
            for token in tokens:
                if token not in self.vocabulary:
                    self.vocabulary[token] = len(self.vocabulary)
        self.inverse_vocabulary = dict(
            (v, k) for k, v in self.vocabulary.items()
        )
    
    def encode(self, text):
        text = self.standardize(text)
        tokens = self.tokenize(text)
        return [self.vocabulary.get(token, 1) for token in tokens]
    
    def decode(self, int_sequence):
        return " ".join(
            self.inverse_vocabulary.get(i, "[UNK]") for i in int_sequence
        )

vectorizer = Vectorizer()
dataset = [
    "I write, erase, rewrite",
    "Erase again, and then",
    "A poppy blooms.",
]
vectorizer.make_vocabulary(dataset)

In [2]:
test_sentence = "I write, rewrite, and still rewrite again"
encoded_sequence = vectorizer.encode(test_sentence)
print(encoded_sequence)

[2, 3, 5, 7, 1, 5, 6]


In [3]:
decoded_sentence = vectorizer.decode(encoded_sequence)
print(decoded_sentence)

i write rewrite and [UNK] rewrite again


In [7]:
from tensorflow.keras.layers import TextVectorization
text_vectorization = TextVectorization(
    output_mode="int",
)

In [5]:
import re
import string
import tensorflow as tf

def custom_standardization_fn(string_tensor):
    lowercase_string = tf.strings.lower(string_tensor)
    return tf.strings.regex_replace(
        lowercase_string, f"[{re.escape(string.punctuation)}]", ""
    )

def custom_split_fn(string_tensor):
    return tf.strings.split(string_tensor)

text_vectorization = TextVectorization(
    output_mode="int",
    standardize=custom_standardization_fn,
    split=custom_split_fn,
)

In [6]:
dataset = [
    "I write, erase, rewrite",
    "Erase again, and then",
    "A poppy blooms.",
]
text_vectorization.adapt(dataset)

In [7]:
text_vectorization.get_vocabulary()

['',
 '[UNK]',
 'erase',
 'write',
 'then',
 'rewrite',
 'poppy',
 'i',
 'blooms',
 'and',
 'again',
 'a']

In [8]:
vocabulary = text_vectorization.get_vocabulary()
test_sentence = "I write, rewrite, and still rewrite again"
encoded_sentence = text_vectorization(test_sentence)
print(encoded_sentence)

tf.Tensor([ 7  3  5  9  1  5 10], shape=(7,), dtype=int64)


In [9]:
inverse_vocabulary = dict(enumerate(vocabulary))
decoded_sentence = " ".join(inverse_vocabulary[int(i)] for i in encoded_sentence)
print(decoded_sentence)

i write rewrite and [UNK] rewrite again


In [17]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -f aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
 17 80.2M   17 13.7M    0     0   143k      0  0:09:30  0:01:37  0:07:53     0:07:34  0:00:54  0:06:40  201k^C

gzip: stdin: unexpected end of file
tar: 아카이브에 예기치 않은 파일 끝 문자
tar: 아카이브에 예기치 않은 파일 끝 문자
tar: Error is not recoverable: exiting now


In [ ]:
!rm -r aclImdb/train/unsup

In [ ]:
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [ ]:
import os, pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

In [1]:
from tensorflow import keras
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)

val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)

test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

2024-02-16 17:30:56.558083: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-16 17:30:56.593600: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-16 17:30:56.593635: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-16 17:30:56.594447: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-16 17:30:56.600664: I tensorflow/core/platform/cpu_feature_guar

Found 20000 files belonging to 2 classes.


2024-02-16 17:30:58.778162: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-16 17:30:58.807946: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-16 17:30:58.808117: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [20]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b"I would love to have that two hours of my life back. It seemed to be several clips from Steve's Animal Planet series that was spliced into a loosely constructed script. Don't Go, If you must see it, wait for the video ...", shape=(), dtype=string)
targets[0]: tf.Tensor(0, shape=(), dtype=int32)


In [3]:
from tensorflow.keras.layers import TextVectorization

text_vectorization = TextVectorization(
    max_tokens=20000,
    output_mode="multi_hot",
)

text_only_train_ds = train_ds.map(lambda x, y: x)
text_vectorization.adapt(text_only_train_ds)

binary_1gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=8,
)
binary_1gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=8,
)
binary_1gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=8,
)

In [ ]:
for inputs, targets in binary_1gram_train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32, 20000)
inputs.dtype: <dtype: 'float32'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32)
targets[0]: tf.Tensor(1, shape=(), dtype=int32)


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

def get_model(max_tokens=20000, hidden_dim=16):
    inputs = keras.Input(shape=(max_tokens, ))
    x = layers.Dense(hidden_dim, activation="relu")(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer="rmsprop",
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    return model

In [23]:
model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_1gram.keras", save_best_only=True)
]
model.fit(
    binary_1gram_train_ds.cache(),
    epochs=10,
    validation_data=binary_1gram_val_ds.cache(),
    callbacks=callbacks,
)
model = keras.models.load_model("binary_1gram.keras")
print(f"테스트 정확도: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_4 (Dense)             (None, 16)                320016    
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320033 (1.22 MB)
Trainable params: 320033 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Epoch 1/10
625/625 [==============================] - 4s 5ms/step - loss: 0.4151 - accuracy: 0.8242 - val_loss: 0.2836 - val_accuracy: 0.8872
Epoch 2/10
625/625 [==============================] - 2s 3ms/step - loss: 0.2745 - accuracy: 0.8980 - val_loss: 0.2692 - val_accuracy: 0.8924
Epoch 3/10
625/625 [==============================] - 2s 3ms/step - loss: 0.2386 - accuracy: 0.9170 - val_loss: 0.2790 - val_accuracy: 0.8984
Epoch 4/10
625/625 [==============================] - 2s 3ms/step - loss: 0.2247 - accuracy: 0.9231 - val_loss: 0.2968 - val_accuracy: 0.8940
Epoch 5/10
625/625 [==============================] - 2s 3ms/step - loss: 0.2174 - accuracy: 0.9288 - val_loss: 0.3044 - val_accuracy: 0.8918
Epoch 6/10
625/625 [==============================] - 2s 3ms/step - loss: 0.2107 - accuracy: 0.9319 - val_loss: 0.3139 - val_accuracy: 0.8920
Epoch 7/10
625/625 [==============================] - 2s 3ms/step - loss: 0.2080 - accuracy: 0.9359 - val_loss: 0.3260 - val_accuracy: 0.8924
Epoch 

In [25]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="multi_hot",
)

In [26]:
text_vectorization.adapt(text_only_train_ds)

binary_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=8,
)

binary_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=8,
)

binary_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=8,
)

model = get_model()
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("binary_2gram.keras", save_best_only=True)
]

model.fit(
    binary_2gram_train_ds,
    epochs=10,
    validation_data=binary_2gram_val_ds,
    callbacks=callbacks,
)

model = keras.models.load_model("binary_2gram.keras")
print(f"테스트 정확도: {model.evaluate(binary_2gram_test_ds)[1]:.3f}")

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_6 (Dense)             (None, 16)                320016    
                                                                 
 dropout_3 (Dropout)         (None, 16)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320033 (1.22 MB)
Trainable params: 320033 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 3s 4ms/step - loss: 0.3728 - accuracy: 0.8464 - val_loss: 0.2629 - val_accuracy: 0.8954
Epoch 2/10
62

In [27]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="count",
)

In [3]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="tf_idf",
)

In [29]:
text_vectorization.adapt(text_only_train_ds)

tfidf_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=8,
)

tfidf_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=8,
)

tfidf_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=8,
)

model = get_model()
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("tfidf_2gram.keras", save_best_only=True)
]

model.fit(
    tfidf_2gram_train_ds,
    epochs=10,
    validation_data=tfidf_2gram_val_ds,
    callbacks=callbacks,
)

model = keras.models.load_model("tfidf_2gram.keras")
print(f"테스트 정확도: {model.evaluate(tfidf_2gram_test_ds)[1]:.3f}")

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_8 (Dense)             (None, 16)                320016    
                                                                 
 dropout_4 (Dropout)         (None, 16)                0         
                                                                 
 dense_9 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320033 (1.22 MB)
Trainable params: 320033 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 3s 5ms/step - loss: 0.4881 - accuracy: 0.7934 - val_loss: 0.3055 - val_accuracy: 0.8756
Epoch 2/10
62

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [6]:
inputs = keras.Input(shape=(1, ), dtype="string")
processed_inputs = text_vectorization(inputs)
outputs = model(processed_inputs)
inference_model = keras.Model(inputs, outputs)

RuntimeError: Exception encountered when calling layer 'string_lookup_1' (type StringLookup).

When using `output_mode=tf_idf` and `pad_to_max_tokens=False`, you must set the layer's vocabulary before calling it. Either pass a `vocabulary` argument to the layer, or call `adapt` with some sample data.

Call arguments received by layer 'string_lookup_1' (type StringLookup):
  • inputs=tf.RaggedTensor(values=Tensor("text_vectorization_1/StringNGrams/StringNGrams:0", shape=(None,), dtype=string), row_splits=Tensor("text_vectorization_1/StringNGrams/StringNGrams:1", shape=(None,), dtype=int64))

In [7]:
import tensorflow as tf

raw_text_data = tf.convert_to_tensor([
    ["That was an excellent movie, I loved it."],
])
predictions = inference_model(raw_text_data)
print(f"긍정적인 리뷰일 확률: {float(predictions[0] * 100):.2f}%%")

NameError: name 'inference_model' is not defined

In [4]:
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

text_only_train_ds = train_ds.map(lambda x, y: x)

max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=8,
)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=8,
)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=8,
)

In [16]:
import tensorflow as tf

inputs = keras.Input(shape=(None, ), dtype="int64")
embedded = tf.one_hot(inputs, depth=max_tokens)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 tf.one_hot_1 (TFOpLambda)   (None, None, 20000)       0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                5128448   
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5128513 (19.56 MB)
Trainable params: 5128513 (19.56 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

In [17]:
callbacks = [
    keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras", save_best_only=True)
]

model.fit(
    int_train_ds,
    validation_data=int_val_ds,
    epochs=10,
    callbacks=callbacks,
)

model = keras.models.load_model("one_hot_bidir_lstm.keras")
print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10


2024-02-15 22:22:56.652434: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-02-15 22:22:57.736422: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fa9641b3e30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-15 22:22:57.736451: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2024-02-15 22:22:57.744293: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1708003377.808500   56078 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


625/625 [==============================] - 114s 177ms/step - loss: 0.5594 - accuracy: 0.7156 - val_loss: 0.3928 - val_accuracy: 0.8550
Epoch 2/10
625/625 [==============================] - 110s 176ms/step - loss: 0.3679 - accuracy: 0.8605 - val_loss: 0.3176 - val_accuracy: 0.8742
Epoch 3/10
625/625 [==============================] - 110s 177ms/step - loss: 0.3001 - accuracy: 0.8945 - val_loss: 0.3370 - val_accuracy: 0.8776
Epoch 4/10
625/625 [==============================] - 110s 177ms/step - loss: 0.2568 - accuracy: 0.9126 - val_loss: 0.3106 - val_accuracy: 0.8832
Epoch 5/10
625/625 [==============================] - 110s 176ms/step - loss: 0.2256 - accuracy: 0.9255 - val_loss: 0.3234 - val_accuracy: 0.8882
Epoch 6/10
625/625 [==============================] - 110s 176ms/step - loss: 0.1969 - accuracy: 0.9348 - val_loss: 0.3165 - val_accuracy: 0.8678
Epoch 7/10
625/625 [==============================] - 110s 176ms/step - loss: 0.1729 - accuracy: 0.9445 - val_loss: 0.3729 - val_accura

In [18]:
embedding_layer = layers.Embedding(input_dim=max_tokens, output_dim=256)

In [19]:
inputs = keras.Input(shape=(None, ), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_lstm.keras", save_best_only=True)
]

model.fit(
    int_train_ds,
    validation_data=int_val_ds,
    epochs=10,
    callbacks=callbacks,
)

model = keras.models.load_model("embeddings_bidir_lstm.keras")
print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional_2 (Bidirecti  (None, 64)                73984     
 onal)                                                           
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5194049 (19.81 MB)
Trainable params: 5194049 (19.81 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

In [20]:
embedding_layer = layers.Embedding(input_dim=10, output_dim=256, mask_zero=True)
some_input = [
    [4, 3, 2, 1, 0, 0, 0],
    [5, 4, 3, 2, 1, 0, 0],
    [2, 1, 0, 0, 0, 0, 0]
]
mask = embedding_layer.compute_mask(some_input)
print(mask)

tf.Tensor(
[[ True  True  True  True False False False]
 [ True  True  True  True  True False False]
 [ True  True False False False False False]], shape=(3, 7), dtype=bool)


In [21]:
inputs = keras.Input(shape=(None, ), dtype="int64")
embedded = layers.Embedding(
    input_dim=max_tokens,
    output_dim=256,
    mask_zero=True,
)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_lstm_with_masking.keras",
                                    save_best_only=True)
]

model.fit(
    int_train_ds,
    validation_data=int_val_ds,
    epochs=10,
    callbacks=callbacks,
)

model = keras.models.load_model("embeddings_bidir_lstm_with_masking.keras")
print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_3 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional_3 (Bidirecti  (None, 64)                73984     
 onal)                                                           
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5194049 (19.81 MB)
Trainable params: 5194049 (19.81 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

2024-02-15 23:29:25.605095: W tensorflow/core/common_runtime/type_inference.cc:339] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}

	for Tuple type infernce function 0
	while inferring type of node 'cond_36/output/_23'


625/625 [==============================] - 49s 71ms/step - loss: 0.4402 - accuracy: 0.7913 - val_loss: 0.3423 - val_accuracy: 0.8382
Epoch 2/10
625/625 [==============================] - 30s 48ms/step - loss: 0.2775 - accuracy: 0.8906 - val_loss: 0.2907 - val_accuracy: 0.8818
Epoch 3/10
625/625 [==============================] - 27s 44ms/step - loss: 0.2177 - accuracy: 0.9179 - val_loss: 0.3116 - val_accuracy: 0.8708
Epoch 4/10
625/625 [==============================] - 28s 45ms/step - loss: 0.1656 - accuracy: 0.9397 - val_loss: 0.3124 - val_accuracy: 0.8810
Epoch 5/10
625/625 [==============================] - 23s 37ms/step - loss: 0.1277 - accuracy: 0.9548 - val_loss: 0.3718 - val_accuracy: 0.8804
Epoch 6/10
625/625 [==============================] - 24s 38ms/step - loss: 0.0977 - accuracy: 0.9665 - val_loss: 0.3955 - val_accuracy: 0.8764
Epoch 7/10
625/625 [==============================] - 22s 35ms/step - loss: 0.0755 - accuracy: 0.9757 - val_loss: 0.4743 - val_accuracy: 0.8698
Epo

In [22]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip
!rm -f glove.6B.zip

--2024-02-15 23:40:55--  http://nlp.stanford.edu/data/glove.6B.zip
nlp.stanford.edu (nlp.stanford.edu) 해석 중... 171.64.67.140
다음으로 연결 중: nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 302 Found
위치: https://nlp.stanford.edu/data/glove.6B.zip [따라감]
--2024-02-15 23:40:55--  https://nlp.stanford.edu/data/glove.6B.zip
다음으로 연결 중: nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 301 Moved Permanently
위치: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [따라감]
--2024-02-15 23:40:56--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
downloads.cs.stanford.edu (downloads.cs.stanford.edu) 해석 중... 171.64.64.22
다음으로 연결 중: downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 200 OK
길이: 862182613 (822M) [application/zip]
저장 위치: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.10MB/s    / 2m 46s   

2024-02-15 23:43

In [23]:
import numpy as np

path_to_glove_file = "glove.6B.100d.txt"
embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print(f"단어 벡터 개수: {len(embeddings_index)}")

단어 벡터 개수: 400000


In [24]:
embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items():
    if i < max_tokens:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [26]:
embedding_layer = layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
    mask_zero=True,
)

In [27]:
inputs = keras.Input(shape=(None, ), dtype="int64")
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("glove_embeddings_sequence_model.keras",
                                    save_best_only=True)
]

model.fit(
    int_train_ds,
    validation_data=int_val_ds,
    epochs=10,
    callbacks=callbacks,
)

model = keras.models.load_model("glove_embeddings_sequence_model.keras")
print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_4 (Embedding)     (None, None, 100)         2000000   
                                                                 
 bidirectional_4 (Bidirecti  (None, 64)                34048     
 onal)                                                           
                                                                 
 dropout_4 (Dropout)         (None, 64)                0         
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2034113 (7.76 MB)
Trainable params: 34113 (133.25 KB)
Non-trainable params: 2000000 (7.63 MB)
_________________

In [5]:
# 데이터 준비
from tensorflow import keras
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)

val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)

test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

text_only_train_ds = train_ds.map(lambda x, y: x)

max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=8,
)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=8,
)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=8,
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim      # 입력 토큰 벡터의 크기
        self.dense_dim = dense_dim      # 내부 밀집 층의 크기
        self.num_heads = num_heads      # 어텐션 헤드의 개수
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim = embed_dim,
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim)]
        )
        self.layernorm_1 = layers.LayerNormalization(epsilon=1e-7)
        self.layernorm_2 = layers.LayerNormalization(epsilon=1e-7)
        
    # call 메소드에서 연산을 수행한다.
    def call(self, inputs, mask=None):
        # Embedding 층에서 생성하는 마스크는 2D이지만 어텐션 층은 3D 또는 4D를 기대하므로 랭크를 높인다.
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask,
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
    
    # 모델을 저장할 수 있도록 직렬화를 구현한다.
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [7]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None, ), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 256)         5120000   
                                                                 
 transformer_encoder_1 (Tra  (None, None, 256)         543776    
 nsformerEncoder)                                                
                                                                 
 global_max_pooling1d_1 (Gl  (None, 256)               0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                 257 

In [8]:
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_encoder.keras", save_best_only=True)
]

model.fit(
    int_train_ds,
    validation_data=int_val_ds,
    epochs=20,
    callbacks=callbacks,
)

model = keras.models.load_model(
    "transformer_encoder.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder}   # 모델 사용 시 사용자 정의 TransformerEncoder 클래스를 지정한다.
)
print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/20


2024-02-16 19:41:48.736001: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f7404943ed0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-16 19:41:48.736032: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2024-02-16 19:41:48.743705: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-16 19:41:48.760317: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
I0000 00:00:1708080108.809391    9184 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


625/625 [==============================] - 40s 58ms/step - loss: 0.5055 - accuracy: 0.7595 - val_loss: 0.3515 - val_accuracy: 0.8458
Epoch 2/20
625/625 [==============================] - 31s 49ms/step - loss: 0.3500 - accuracy: 0.8512 - val_loss: 0.3173 - val_accuracy: 0.8658
Epoch 3/20
625/625 [==============================] - 29s 47ms/step - loss: 0.3111 - accuracy: 0.8666 - val_loss: 0.3108 - val_accuracy: 0.8738
Epoch 4/20
625/625 [==============================] - 28s 45ms/step - loss: 0.2833 - accuracy: 0.8828 - val_loss: 0.3028 - val_accuracy: 0.8738
Epoch 5/20
625/625 [==============================] - 28s 45ms/step - loss: 0.2550 - accuracy: 0.8929 - val_loss: 0.2984 - val_accuracy: 0.8808
Epoch 6/20
625/625 [==============================] - 28s 44ms/step - loss: 0.2284 - accuracy: 0.9087 - val_loss: 0.3162 - val_accuracy: 0.8704
Epoch 7/20
625/625 [==============================] - 27s 44ms/step - loss: 0.1942 - accuracy: 0.9227 - val_loss: 0.2979 - val_accuracy: 0.8834
Epo

In [7]:
class PositionalEmbedding(layers.Layer):
    # 위치 임베딩의 단점은 시퀀스 길이를 미리 알아야 한다는 것이다.
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        # 토큰 인덱스를 위한 Embedding 층을 준비한다.
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim,
        )
        # 토큰 위치를 위한 Embedding 층을 준비한다.
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim,
        )
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim
    
    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        # 두 임베딩 벡터를 더한다.
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)
    
    # 모델 저장을 위한 직렬화를 구현한다.
    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [8]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None, ), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("full_transformer_encoder.keras",
                                    save_best_only=True)
]

model.fit(
    int_train_ds,
    validation_data=int_val_ds,
    epochs=20,
    callbacks=callbacks,
)

model = keras.models.load_model(
    "full_transformer_encoder.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding}
)
print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 positional_embedding_1 (Po  (None, None, 256)         5273600   
 sitionalEmbedding)                                              
                                                                 
 transformer_encoder_1 (Tra  (None, None, 256)         543776    
 nsformerEncoder)                                                
                                                                 
 global_max_pooling1d_1 (Gl  (None, 256)               0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                           

2024-02-17 16:35:27.655032: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f67981f6900 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-17 16:35:27.655077: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2024-02-17 16:35:27.662883: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-17 16:35:27.681217: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
I0000 00:00:1708155327.733506   52877 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


625/625 [==============================] - 43s 63ms/step - loss: 0.5159 - accuracy: 0.7467 - val_loss: 0.3898 - val_accuracy: 0.8158
Epoch 2/20
625/625 [==============================] - 34s 54ms/step - loss: 0.2954 - accuracy: 0.8741 - val_loss: 0.2788 - val_accuracy: 0.8876
Epoch 3/20
625/625 [==============================] - 31s 50ms/step - loss: 0.2336 - accuracy: 0.9046 - val_loss: 0.3839 - val_accuracy: 0.8450
Epoch 4/20
625/625 [==============================] - 31s 50ms/step - loss: 0.1906 - accuracy: 0.9259 - val_loss: 0.2853 - val_accuracy: 0.8916
Epoch 5/20
625/625 [==============================] - 32s 50ms/step - loss: 0.1622 - accuracy: 0.9376 - val_loss: 0.4403 - val_accuracy: 0.8714
Epoch 6/20
625/625 [==============================] - 32s 50ms/step - loss: 0.1374 - accuracy: 0.9477 - val_loss: 0.3832 - val_accuracy: 0.8588
Epoch 7/20
625/625 [==============================] - 31s 50ms/step - loss: 0.1150 - accuracy: 0.9566 - val_loss: 0.4561 - val_accuracy: 0.8776
Epo

In [9]:
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
!unzip -q spa-eng.zip
!rm -f spa-eng.zip

--2024-02-17 17:43:01--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
storage.googleapis.com (storage.googleapis.com) 해석 중... 34.64.4.91, 34.64.4.123, 34.64.4.27, ...
다음으로 연결 중: storage.googleapis.com (storage.googleapis.com)|34.64.4.91|:80... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 200 OK
길이: 2638744 (2.5M) [application/zip]
저장 위치: ‘spa-eng.zip’

spa-eng.zip         100%[===================>]   2.52M  --.-KB/s    / 0.07s    

2024-02-17 17:43:02 (37.0 MB/s) - ‘spa-eng.zip’ 저장함 [2638744/2638744]



In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
text_file = "spa-eng/spa.txt"
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    english, spanish = line.split("\t")
    spanish = "[start] " + spanish + " [end]"
    text_pairs.append((english, spanish))

In [4]:
import random
print(random.choice(text_pairs))

('The smell was offensive.', '[start] El olor era desagradable. [end]')


In [5]:
import random

random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

In [6]:
import tensorflow as tf
import string
import re

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", ""
    )

vocab_size = 15000
sequence_length = 20

# 영어 층
source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
# 스페인어 층
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,     # 훈련하는 동안 한 스텝 앞선 문장이 필요하기 때문에 토큰 하나가 추가된 스페인어 문장을 생성한다.
    standardize=custom_standardization,
)

train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
# 각 언어의 어휘 사전을 만든다.
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

2024-02-18 00:32:11.790466: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-18 00:32:11.790675: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-18 00:32:11.790778: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [7]:
batch_size = 64

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
        "english": eng,
        "spanish": spa[:, :-1],     # 입력 스페인어 문장은 마지막 토큰을 포함하지 않기 때문에 입력과 타깃 길이가 같다.
    }, spa[:, 1:])                  # 타깃 스페인어 문장은 한 스텝 앞의 문장이다. 길이는 입력과 같다.

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=8)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [8]:
for inputs, targets in train_ds.take(1):
    print(f"inputs[\"english\"].shape: {inputs['english'].shape}")
    print(f"inputs[\"spanish\"].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs["english"].shape: (64, 20)
inputs["spanish"].shape: (64, 20)
targets.shape: (64, 20)


2024-02-18 00:32:47.163082: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [9]:
inputs = keras.Input(shape=(sequence_length, ), dtype="int64")
x = layers.Embedding(input_dim=vocab_size, output_dim=128)(inputs)
x = layers.LSTM(32, return_sequences=True)(x)
outputs = layers.Dense(vocab_size, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [10]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 1024

source = keras.Input(shape=(None, ), dtype="int64", name="english")     # 영어 소스 문장이 여기에 입력된다. 입력 이름을 지정함으로써 입력 딕셔너리로 모델을 훈련할 수 있다.
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)     # 마스킹은 이 방식에서 중요하다.
encoded_source = layers.Bidirectional(
    layers.GRU(latent_dim), merge_mode="sum"
)(x)    # 인코딩된 소스 문장은 양방향 GRU의 마지막 출력이다.

In [11]:
past_target = keras.Input(shape=(None, ), dtype="int64", name="spanish")    # 스페인어 타깃 시퀀스가 여기에 입력된다.
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)    # 마스킹은 필수이다.
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)                            # 인코딩된 소스 시퀀스는 디코더 GRU의 초기 상태가 된다.
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)        # 다음 토큰을 예측한다.
# 엔드-투-엔드 모델은 소스 시퀀스와 타깃 시퀀스를 한 스텝 앞의 타깃 시퀀스에 매핑한다.
seq2seq_rnn = keras.Model(inputs=[source, past_target], outputs=target_next_step)

In [12]:
seq2seq_rnn.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

seq2seq_rnn.fit(
    train_ds,
    epochs=15,
    validation_data=val_ds,
)

Epoch 1/15


2024-02-18 00:32:57.785105: W tensorflow/core/common_runtime/type_inference.cc:339] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}

	for Tuple type infernce function 0
	while inferring type of node 'cond_35/output/_22'
2024-02-18 00:32:58.496365: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-02-18 00:32:59.527668: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f7682865a10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-18 00:32

1302/1302 [==============================] - 88s 61ms/step - loss: 4.6846 - accuracy: 0.3193 - val_loss: 3.9046 - val_accuracy: 0.3865
Epoch 2/15
1302/1302 [==============================] - 72s 55ms/step - loss: 3.7190 - accuracy: 0.4167 - val_loss: 3.2981 - val_accuracy: 0.4628
Epoch 3/15
1302/1302 [==============================] - 75s 58ms/step - loss: 3.2073 - accuracy: 0.4739 - val_loss: 2.8680 - val_accuracy: 0.5182
Epoch 4/15
1302/1302 [==============================] - 73s 56ms/step - loss: 2.8477 - accuracy: 0.5148 - val_loss: 2.6100 - val_accuracy: 0.5536
Epoch 5/15
1302/1302 [==============================] - 72s 55ms/step - loss: 2.5733 - accuracy: 0.5477 - val_loss: 2.4218 - val_accuracy: 0.5813
Epoch 6/15
1302/1302 [==============================] - 73s 56ms/step - loss: 2.3515 - accuracy: 0.5756 - val_loss: 2.3014 - val_accuracy: 0.5988
Epoch 7/15
1302/1302 [==============================] - 72s 55ms/step - loss: 2.1632 - accuracy: 0.6002 - val_loss: 2.2022 - val_accura

In [13]:
import numpy as np

# 예측된 인덱스를 문자열 토큰으로 변환하기 위한 딕셔너리를 준비한다.
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"        # 시드 토큰
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])
        # 다음 토큰을 샘플링한다.
        next_token_predictions = seq2seq_rnn.predict([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(next_token_predictions[0, i, :])
        # 다음 토큰 예측을 문자열로 바꾸고 생성된 문장에 추가한다.
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += (" " + sampled_token)
        if sampled_token == "[end]":    # 종료 조건: 최대 길이에 도달하거나 종료 문자가 생성된 경우
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
This is nonsense.
1/1 [==============================] - 0s 19ms/step
[start] esto es [UNK] [end]
-
You'll learn how to do it sooner or later.
1/1 [==============================] - 0s 19ms/step
[start] aprenderás a hacerlo tarde o temprano [end]
-
They haven't come back home yet.
1/1 [==============================] - 0s 19ms/step
[start] todavía no han venido de casa [end]
-
Sorry for the harsh words.
1/1 [==============================] - 0s 19ms/step
[start] [UNK] por el proyecto de palabras [end]
-
I'm leaving on Sunday.
1/1 [==============================] - 0s 20ms/step
[start] me voy a la playa [end]
-
I thought it'd be more comfortable if we sat here.
1/1 [==============================] - 0s 20ms/step
[start] pensé que sería más tiempo para que nos [UNK] aquí [end]
-
Tom is proud of his daughter.
1/1 [==============================] - 0s 20ms/step
[start] tom está orgulloso de su hija [end]
-
We are out of danger.
1/1 [==============================] - 0s 19ms/step
[start] 